In [31]:
import numpy as np


In [32]:
def calculos(MN, MC, K, s, r, U, F):

    n = len(MN) #numero de nodos
    gl = 2 # grados de libertad
    MR_global = np.zeros([gl*n,gl*n])
    # un 0 en la matriz F significa que al moverse en esa direccion no hay resistencia

    for e in range(n):
        # este if es para el ultimo elemento, le resta al ultimo elemento el primer elemento
        if e==(n-1):
            delta_x = MN[0][0] - MN[e][0]
            delta_y = MN[0][1] - MN[e][1]
            angulo = np.arctan2(delta_x, delta_y) # para ver en que cuadrante estoy y no tener problemas de signo
            C = np.cos(angulo)
            S = np.sin(angulo)
            MR_local = np.array([(C**2,C*S,-C**2,-C*S),(C*S,S**2,-C*S,-S**2),(-C**2,-C*S,C**2,C*S),(-C*S,-S**2, C*S,S**2)])
            for i in MC[e]:
                for j in MC[e]:
                    columna_i = list(MC[e]).index(i) # identifico en que columna de MC esta el elemento i
                    columna_j = list(MC[e]).index(j) # # identifico en que columna de MC esta el elemento i
                    MR_global[i*gl:(i+1)*gl , j*gl:(j+1)*gl] += K[e] * (MR_local[columna_i*gl:(columna_i+1)*gl , columna_j*gl:(columna_j+1)*gl])


        else:
            delta_x = MN[e+1][0] - MN[e][0]
            delta_y = MN[e+1][1] - MN[e][1]
            angulo = np.arctan2(delta_y, delta_x) # para ver en que cuadrante estoy y no tener problemas de signo
            C = np.cos(angulo)
            S = np.sin(angulo)
            MR_local = np.array([(C**2,C*S,-C**2,-C*S),(C*S,S**2,-C*S,-S**2),(-C**2,-C*S,C**2,C*S),(-C*S,-S**2, C*S,S**2)])

            for i in MC[e]:
                columna_i = list(MC[e]).index(i) # identifico en que columna de MC esta el elemento i
                for j in MC[e]:
                    columna_j = list(MC[e]).index(j) # # identifico en que columna de MC esta el elemento i
                    MR_global[i*gl:(i+1)*gl , j*gl:(j+1)*gl] += K[e] *(MR_local[columna_i*gl:(columna_i+1)*gl , columna_j*gl:(columna_j+1)*gl])
                    

    scale = np.max(np.max(MR_global))
    #            MR_global[abs(MR_global[i][j]/scale) < 1e-9] = 0
    # esta linea de arriba no me funciona, por eso hago el doble for
    for i in range(len(MR_global)):
        for j in range(len(MR_global)):
            if abs(MR_global[i][j]) < 1e-9:
                MR_global[i][j] = 0


    # determinante = 0, deja de ser 0 cuando se usa la matriz reducida
    MR_reducida = MR_global.copy()
    # borro filas y columnas de las posiciones de U que ya tengo
    MR_reducida = np.delete(MR_reducida, s, axis=0)
    MR_reducida = np.delete(MR_reducida, s, axis=1)

    nuevas_U = np.linalg.solve(MR_reducida, F) # obtengo las U que no conozco


    # armo el vector U completo
    U_global = np.zeros(n*gl)
    k=0
    z=0
    # k y z son contadores que van recorriendo en orden los vectores U y nuevas_U

    for i in range(len(U_global)):
        if i in s:
            U_global[i] = U[k]
            k += 1

        if i in r:
            U_global[i] = nuevas_U[z]
            z += 1

    # ahora obtengo el vector F completo
    F = np.matmul(MR_global,U_global)

    return(F, U_global, MR_global)


In [33]:
## EJEMPLO MENSULA
MN = np.array([(0,0), (1,0), (1,1)])# matriz_nodos: matriz con las posiciones, en cada fila estan los nodos y cada columna las coordenadas x,y,z, etc
MC = np.array([(0,1), (1,2), (2,0)]) # matriz_conectividad: fila cada element, columna cada nodo que forman ese elemento
K = [10,5,20] # vector con las constantes elasticas de cada elemento
s = [0,1,3] # posiciones de los desplazamientos que conozco
r = [2,4,5] # posiciones de las fuerzas que conozco
U = [0,0,0] # valores de los desplazamientos, respectivamente a s
F = [0,2,1] # valores de las fuerzas, respectivamente a r

In [34]:
F, U, matriz_rigidez = calculos(MN, MC, K, s, r, U, F)

print('Fuerzas =', F)
print('Desplazamientos =', U)

Fuerzas = [-2. -2.  0.  1.  2.  1.]
Desplazamientos = [ 0.   0.   0.   0.   0.4 -0.2]


In [35]:
# EJERCICIO 2 GUIA
MN = [[0,0] , [800,400] , [1600,0] , [800,0]] # en mm
MC = [[0,1] , [3,0] , [3,1] , [1,2], [2, 3]]


A = [1000, 1000, 2000, 1000, 1000] # area en mm2
E = 210 * np.ones(len(MC)) # Gpa = N/mm2
l0 = np.sqrt(800**2 + 400**2)
l3 = np.sqrt(800**2 + 400**2)
L = [l0,800,400,l3,800 ]

K = np.zeros(len(MC))
for i in range(len(K)):
    K[i] = (E[i]*A[i])/L[i]

U = [0,0,0]
s = [0,1,5]

F = [0,0,0,0,-20000] # en Newton
r = [2,3,4,6,7]

In [36]:
F, U, matriz_rigidez = calculos(MN, MC, K, s, r, U, F)

print('Fuerzas =', F)
print('Desplazamientos =', U)

LinAlgError: Singular matrix